<a href="https://colab.research.google.com/github/tonytani37/stats/blob/main/stats_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 前提条件
Googleアカウントが必須です。

ブラウザは最新のChromeの利用を強く推奨します。(Microsoft Edgeの最新版は大丈夫ですが、古いEdgeやIEは動かないかもしれません。FirefoxやSafariでは動くと思いわれます）

## 注意事項
### パソコンが古くても、処理自体は膨大な能力を持つGoogleクラウド上で実行されるので、目の前のPCが多少古くてもあまり心配することはありません。

１．説明を読んで、上から順番に番号順に処理をしてください。

２．処理はそれぞれの機能ごとに分けてありますので、今後修正したい場合にはその部分だけ差し替えればオーケーです。

３．処理によっては実行するとその下によくわからない文字がずらずらと表示されるものとされないものがあります。どのタイミングで処理が終わったかわかりづらいかもしれないので、各処理が完了すると「x おわり」と表示するようにしていますで、そこまで我慢してください。（表示されるメッセージのnは処理の番号です）




### データ化したいPDFファイルを入手します
1-1 は自分のPCにあるPDFファイルを変換したい場合に使います。

1-2 はよくわからないけどとりあえず練習用に試してみたい人向けに、Xリーグのホームページにある2019/8/24の富士通フロンティアーズ対IBM BigBlueの対戦のものがダウンロードされます。

どちらか一つ、好きな方を実行してください。（1-1の場合は、以降使われるPDFファイル名や画像データ名が以下にあるものと異なるので、適宜変更してくださいね）

その１）自分のPCにあるPDFファイルを変換したい場合

In [ ]:
# 1-1
#OCRであつかうPDFファイルを読み込む（変換したいPC上のファイルを指定してくだされ）
from google.colab import files
f = files.upload()
#読み込んだ画像のファイル名
filename = list(f)[0]

print('1-1 おわり')

その２）とりあえず試してみたい場合　

In [ ]:
# 1-2
#OCRであつかうPDFファイルを読み込む（練習用に特定のPDFファイルをダウンロードしてくる）
! wget -c -P '/content/' 'https://xleague.s3.ap-northeast-1.amazonaws.com/wp-content/uploads/2019/07/20190824_X1_FF_BB_921.pdf'
filename = "20190824_X1_FF_BB_921.pdf"
print('1-2 おわり')

### PDFから画像へ変換します

In [ ]:
# 2
# PDFから画像へ変換するためのライブラリ、ツールを準備する
!apt-get install poppler-utils 
!pip install pdf2image
print('2 おわり')

In [ ]:
# 3
# 画像データに変換する処理
from pathlib import Path
from pdf2image import convert_from_path

# Google Colabのデフォルト環境内にimageというフォルダを作る
!rm -r /content/image/
!mkdir /content/image/

pdf_path = Path(filename)
#outputのファイルパス（この場合は上でつくったimageフォルダを指定）
img_path=Path("/content/image/")

#変換されたjpegファイルが保存されます
convert_from_path(pdf_path, output_folder=img_path,fmt='jpeg',output_file=pdf_path.stem)
print('3 おわり')

### 以上で画像変換完了。PDFに複数ページがある場合、ページごとに画像データ（ファイル名末尾が連番）が作成されるので、一つのPDFファイルから複数の画像ファイルが作成されたとしても、驚かないこと。

### ここからOCR読み込みについて。まずは準備しましょう
まずはOCR読み込み用のライブラリとツールをインストールします

In [ ]:
# 4
# Tesseract OCRの関連ライブラリを準備する
# OCR読み込みに必要なツールをインストールします
!apt install tesseract-ocr
!apt install libtesseract-dev
!pip install pyocr
print('4 おわり')

In [ ]:
 # 5
 #日本語トレーニングデータのダウンロードとして環境設定します
!curl -L https://github.com/tesseract-ocr/tessdata/raw/master/jpn.traineddata > jpn.traineddata
!cp jpn.traineddata /usr/share/tesseract-ocr/4.00/tessdata
print('5 おわり')

In [ ]:
# 6
# PythonでOCR読み込み変換に必要なライブラリを準備します
import pyocr
import pyocr.builders
print('6 おわり')

### OCR読み込みの前に、必要なツールがインストールされているか確認しませう。
うまくいかない場合は環境構築が間違っているので、最初からもう一度がんばろう。「ワシはPython先生だから」と自信満々な人は、7,8を飛ばしてもオーケー。

OCR tool is 'Tesseract (sh)' と表示されればオーケー

In [ ]:
# 7
#OCRが使用可能かの設定状態チェックしてみる
tools = pyocr.get_available_tools()
if len(tools) == 0:
    print("OCR tool is not found")
    sys.exit(1)

#OCRツール名を表示
tool = tools[0]
print("OCR tool is '%s'" % (tool.get_name()))
print('7 おわり')

Available languages:　の次に　jpn　が入ってればオーケー（その他 osd,engなどもあるはず。もしかするとないものもあるかもしれないけど、とりあえず jpn が入っていればオーケー）

In [ ]:
# 8
#OCR対応言語を表示して'jpn'が入っているか確認してみる
langs = tool.get_available_languages()
print("Available languages: %s" % ", ".join(langs))
lang = langs[0]
print("Will use lang '%s'" % (lang))
print('8 おわり')

### ここから実際のOCR読み込み処理です。
ここでは、fnameという変数にファイルのパスをセットしています。Google Colaboでは画面左側に作成されたファイルを扱うので、デフォルトの'/content/以降の部分に読み込みたい画像ファイル名(pathも正しく指定してね)を指定してください。ファイル名とパスの指定は、読み込みたいファイルを右クリックすると「パスをコピー」という項目があるので、そいつをクリックしてコピーし、fname = 以降に”または’のどちらかで囲って、その間にペーストすればオーケーです。

(本当は特定のフィルだにある画像ファイルを一括で処理できればいいんだろうけれど、そういうやつは次に考えます）

In [ ]:
# 9
#読み込んだ画像をOCRでテキスト抽出。

#読み込む画像ファイルを指定する（自分でPDFを指定した場合は、このファイル名を変更すること）
fname = '/content/image/20190824_X1_FF_BB_9210001-06.jpg'

txt = tool.image_to_string(
    Image.open(fname),
    lang="jpn",
    builder=pyocr.builders.TextBuilder(tesseract_layout=6)
    )

# プレーに関係ありそうな文字として　"&",'Penalty','Kick-off','Extra Point','TIMEOUT','Quarter'が含まれる行だけ抽出する
# そのほかの特定文字も抽出対象としたい場合は ifの次に or ('' & in line) を追加挿入し、''の間に含めたい文字列を指定するとできますよ
lines_strip = [line for line in txt.splitlines() if ('&' in line) or ('Penalty' in line) or ('Kick-off' in line) or ('Extra Point' in line) or ('TIMEOUT' in line) or (':' in line) or ('Quarter' in line)]
print('9 おわり')

### 私の環境ではなんだか数字の読み取り変換がうまくいってない様子なので、マル付き数字をむりやり半角数字（str型）に置き換える。同時に文字間の不規則な空白をすべて左詰めにする
うまくいっている場合にこれを動かしても（あえてマル付き数字を使っている場合以外は）なんの影響もないはずです。

またここでは文字化したときに不規則に挿入されてしまっている文字間の空白を埋める処理もしていますので、以降の正規化文字判定などで役に立つと思います。

なので、とりあえず動かしてみてください。



In [ ]:
# 10
# 置き換わらないマル数字（やその他の文字）が出てきたら、conv_text辞書に追加してね。形式は辞書のkey:valueです）

# 置き換えたい文字をkey:value形式で定義している辞書形式のデータです
conv_text = {"①":"1","②":"2","③":"3","④":"4","⑤":"5","⑥":"6","⑦":"7","⑧":"8","⑨":"9",
             "⑩":"10","⑪":"11","⑫":"12","⑬":"13","⑭":"14","⑮":"15",
             "⑯":"16","⑰":"17","⑱":"18","⑲":"19","⑳":"20"}

# 文字置き換え用の関数：渡された引数（文字列）に対してkeyと同じ文字がconv_textあったらvalueに置き換える処理を、conv_textの頭から順番にやっとります
def num_conv(line):
    for ct in conv_text:
        line = line.replace(ct,conv_text[ct])
    return line

# 変換された元のデータを一行づつ関数を呼び文字を置き換えた後、スペースを削って左へシフトさせてリスト形式のデータにしています
lines_strip_after =  [''.join(a.split(' ')) for a in [num_conv(line) for line in lines_strip]]

# 変換した結果を試し打ち（アタマの大会名とケツ２行の対戦チームの成績は表示しとりません
for a in lines_strip_after[1:-2]:
    print(a)

print('10 おわり')

### うまく文字化されていましたか？
ここまでの処理で　

PDFファイルの準備→ページごとに画像データ化→画像データから文字抽出→プレーに関係ありそうな行だけ抜きだし　

が完了しました。PDFファイルや画像データと比較してみて、どれくらいうまく処理されたか確認しましょう。変な文字に変換されるときは、まずはPDFファイルや画像データ自体の精度が低いことが考えられますので、それらの対応は各自ググってみてください。（日本語変換用のトレーニングデータをブラッシュアップするという手もありますので、できる人はチャレンジして成功したら結果を広く公開してください）

以降、抜き出した文字データをそれぞれ意味がある項目に分割しなければなりません。これは「正規表現」をつかって意味がある文字を抜き出す処理となります。

### おまけ
PDFからデータ化した結果を、自分のPCにダウンロードできますよ。

注意）Google Colaboは一定時間ほったらかしておくと、せっかく作ったデータが消えてしまいます。（処理プログラム自体は自動的にセーブされているので安心）　データを残したい場合は必要なデータをダウンロードするか、保存場所をGoogle ColaboからGoogle Driveへ変更してください。(Google Driveをマウントする方法などは、各自ググってください。親切な人たちが書いたたくさんの事例が見つかります）

In [ ]:
# OCR処理をした結果文を出力したいときに使う（左側にstats.txtという名前のファイルができる）

# 同じファイル名で繰り返し処理をすると上書きされるので、複数ファイルを扱うとkにはtxt_nameにファイル名を指定してね
txt_name = "stats01.txt"

from google.colab import files
import pandas as pd

# いろいろいじくってリストにしたのでpandasを使って出力する
df = pd.DataFrame(lines_strip_after)
df.to_csv(txt_name,header=False,index=False)


# 続いて作成されたファイルをダウンロードしようとします
# ダウンロードしたくない場合には以下の行をコメントにするか、保存画面が出てきたらキャンセルするとかしてね
files.download(txt_name)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### おまけのおまけ
この処理を上から番号順に実行させればGoogle Colabo環境(中身はJupyter notebook)でPDFデータから文字を抜き出す処理まではできるようになっています。

で、データ分析のためのデータ整備の本番はこれからになりますが、この時点ですでに印刷された文字をデータ化できているので、ファイルをダウンロードした後に、自分で項目を分割してやればエクセルなどパソコン上でデータとして扱えるようになります。

処理を自動化することは作業の効率化という点で非常に大切ですが、意味が分からずやるよりも最初はデータ構造を（なんとなくでも）理解しながら作業を行った方が、後々データの見直しや分析のやり方を見直したいときのアイディアがひらめく可能性が高くなると思います。

今回ダウンロードできるファイル(stats01.txt)はパソコンのメモ帳で開けば、普通の文字列として読んだり修正したりすることができます。分割したい項目をカンマ(,)で区切ってあげて保存した後にファイル名の拡張子をCSVにすれば、それはもう立派なCSVデータです。（CSVは　Comma-Separated　Valuesの略で、文字通りカンマで項目が区切られているデータというだけの意味です。拡張子自体はパソコン自体が処理を楽にするためにつけられている文字列というだけなので、自分が処理させたいプログラムに紐づけられたものに勝手に変えてもオーケーです）

## とりあえず今回の最後に
現在、機械学習などが活用されその精度は上がってきてはいるものの、OCR処理で読み取り間違い発生をゼロにすることはなかなか難しい状況です。特に今回のような数字と記号、漢字とアルファベットが混ざったものをプログラム処理だけですべてきれいなデータが作成できないと考えた方が良いと思います。（元データ自体、半角と全角の数字が混ざっているかもしれないし）

そのため、結局最後は人間が目で確認しながら手で微調整をする必要があります。正直な気持ちとしては今回はそれほど大きいデータ量ではないので、初めから手入力でCSVファイルを作成したほうが良いかもとも思います。しかし、一からデータ登録をすることを考えるとやる気がなくなってしまうので、こういった下準備が気軽にできる環境が生み出されてきていることには大きな意義があると思います。

だだそれ以前に、データ提供されている側にはPDFデータを作るためにエクセルなどで準備しているんじゃないかと思われ、ならばそのデータを（PDFの参考データとしてでも）公開してくれれば、少なくともOCR処理での読み取り間違いなどの発生はなくなり、よりデータを活用しやすくなります。

しかしながら、現在のデータではPASSやPUNTなどの投げたりけったりした選手と、それを受けたりリターンしたりした選手の区別などデータの整理整頓（標準化）ができておらず、突っ込んだ分析や機械学習にこのデータを使おうとした場合にはまだまだ問題は山積状態です。

そんな五里霧中な状態ではあるものの、我々にはNFLという偉大な先達がおられます。彼らがこれまで積み上げてきたノウハウの端っこでも参考にしながら、もっとデータ活用できる環境が構築されることの切望します。

※もしトップレベルのチームではそういったデータ取得、活用ノウハウが共通化されているならば、それについて教えてください。

一度に理想まで近づくことはできませんが、まずは手始めに野球のスコアブックのようなデータ採取の標準フォーマットを決めて、それが各チームで広く使われるようにすることが必須でしょう。そうすれば、それを基にしたツール開発や、データ自体の流通や活用が円滑に行わえるようになると考えます。

これからもっと「データドリブン（データをもとに物事を進めてゆく）」な世の中になると思います。身近なことからコツコツと、できることからやってみましょう。